In [1]:
#==============================================================================================
# 원래 vocab 파일에 신규 vocab 파일에 대해 중복제거해서 추가하기
#==============================================================================================
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk import FreqDist
import numpy as np
import torch
from transformers import BertTokenizer

In [2]:
# new_vocab 파일을 불러옴.
new_vocab_out = '../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab-42000-1/vocab.txt'

# 2차 불용어 정의
#stopwords=['##다', '##하', '있', '##고', '##로', '##한', '##적', '##되', '##었', '##_']
stopwords=['']

new_vocab = []
with open(new_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')

for word in tqdm(data):
    if not word in stopwords:  # 2차 불용어는 제외
        new_vocab.append(word)
        
print(new_vocab[0:10])

  0%|          | 0/37010 [00:00<?, ?it/s]

['으로', '에서', '19', '20', '한다', '201', '200', '에게', '까지', '대한']


In [3]:
# 원래 vocab 리스트로 읽어옴
#org_vocab_out = './moco-vocab/mdistilbertV1.1/vocab.txt'
org_vocab_out = '../../data11/model/distilbert/distilbert-base-multilingual-cased/vocab.txt'
#org_vocab_out = '../../data11/my_corpus/vocab/bert-re-moco-corpus2-nouns-vocab-12000/vocab.txt'

org_vocab = []

with open(org_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')
    
for vocab in tqdm(data):
     org_vocab.append(vocab)

print(len(org_vocab))


  0%|          | 0/119547 [00:00<?, ?it/s]

119547


In [4]:
import re
add_max_vocab_len = 20000 # 새롭게 추가할 vocab 수 

# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거

# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거
#temp = list(set(new_vocab) - set(org_vocab)) #순서 보존이 안됨
    
#또는
# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거print(org_vocab[1111:1115])
print('중복제거 시작==>')
s = set(org_vocab)
temp = [x for x in tqdm(new_vocab) if x not in s] #순서 보존됨    

vocablist = org_vocab + temp[:add_max_vocab_len]  #원본 리스트 + 중복제가한 신규리스트 2개의 리스트를 합침

print(len(vocablist))
print(vocablist[1111:1115])

중복제거 시작==>


  0%|          | 0/37010 [00:00<?, ?it/s]

139547
['વ', 'શ', 'ષ', 'સ']


In [5]:
# 리스트를 파일로 저장
outfilepath = '../../data11/my_corpus/vocab/add_bert_vocab.txt'
with open(outfilepath, 'w') as f:
    f.write('\n'.join(vocablist[:159547])) #159547개만 저장
    #f.write('\n'.join(vocablist))  #모두 저장
    
print('vocab 파일"{}" 추가 성공!. 출력 vocab 파일 : "{}"'.format(new_vocab_out, outfilepath))

vocab 파일"../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab-42000-1/vocab.txt" 추가 성공!. 출력 vocab 파일 : "../../data11/my_corpus/vocab/add_bert_vocab.txt"


In [6]:
from transformers import BertTokenizer

#outfilepath = './moco-vocab/moco-corpus-kowiki2022-nouns-42000-Deduplication-add-en-159552.txt'
# BertTokenizer 로 새롭게 저장된 vocalfile 불러옴
tokenizer = BertTokenizer(vocab_file=outfilepath, 
                          strip_accents=False, 
                          do_lower_case=False)


In [7]:
add_special_token = False

if add_special_token:
    # 옵션 : 저장된 add_new_vocab.txt를 불러와서 special 토큰 추가함
    special_tokens=['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]',
                    '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]',]

    special_tokens_dict = {'additional_special_tokens': special_tokens}
    tokenizer.add_special_tokens(special_tokens_dict)

    # special token 체크
    print(tokenizer.all_special_tokens)

In [8]:
# special token 추가한 special vocab을 저장함
import os
OUT_PATH = '../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab/'
#OUT_PATH = './mdistilbertV2.2'
os.makedirs(OUT_PATH, exist_ok=True)
tokenizer.save_pretrained(OUT_PATH)

('../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab/tokenizer_config.json',
 '../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab/special_tokens_map.json',
 '../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab/vocab.txt',
 '../../data11/my_corpus/vocab/bert-re-kowiki-202206-bert-mecab/added_tokens.json')